<a href="https://colab.research.google.com/github/unihwan/punctuation_restoration/blob/main/nlp_norm2unnorm_keras_char_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#for google drive directory (colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Change your path location here 

In [ ]:
filepath = ""

In [ ]:
#Importing library 
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense,Bidirectional,Activation,dot,concatenate,TimeDistributed
from keras.initializers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import os
import math
import re
import pickle

In [ ]:
# function to get tokenizer
def tokenize(lang):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', lower=False, num_words=None, char_level=True)
    tokenizer.fit_on_texts(lang)
    return tokenizer

In [ ]:
#PREPROCESSING
#get line by line from training file
with open(filepath + '2008.txt', 'r',encoding='cp1252') as f:
    #seperate them line by line
    lines = f.read().split('\n')

In [ ]:
#initialization
# input_texts =  normalized texts
# target_texts = unnormalized texts 
input_texts = []
target_texts = []

In [ ]:
#For each line all the character is lowered
#remove punctuation
#added \t and \n for decoder
for line in lines:
     input_text = line.lower() #turn line into small capital
     input_text = re.sub(r"[^a-zA-Z0-9]+", " ", input_text) #remove punctuation
     target_text = '\t' + line + '\n' #added \t for start and \n indication

     #add normalized text into input data and unnormalized data into target data
     input_texts.append(input_text) 
     target_texts.append(target_text)

In [ ]:
#tokenize input text and put in reverse dic
enc_token = tokenize(input_texts)
reverse_enc_dict = {i: c for c, i in enc_token.word_index.items()}

#tokenize target text and put in reverse dic
dec_token = tokenize(target_texts)
reverse_dec_dict = {i: c for c, i in dec_token.word_index.items()}

In [ ]:
#Number of token add 1 for padding
num_encoder_tokens = len(enc_token.word_index)+1
num_decoder_tokens = len(dec_token.word_index)+1
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
#Print data
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 562574
Number of unique input tokens: 38
Number of unique output tokens: 114
Max sequence length for inputs: 1428
Max sequence length for outputs: 1493


In [ ]:
# save in pickle file
save_file = {'max_encoder_seq_length': max_encoder_seq_length,
            'max_decoder_seq_length': max_decoder_seq_length,
            'num_encoder_tokens': num_encoder_tokens,
            'num_decoder_tokens': num_decoder_tokens,
            'reverse_enc_dict': reverse_enc_dict,
            'reverse_dec_dict': reverse_dec_dict,
            'enc_token': enc_token,
            'dec_token': dec_token}

with open(filepath + 'params_char.pkl', 'wb') as handle:
   pickle.dump(save_file, handle, protocol=pickle.HIGHEST_PROTOCOL)

Number of samples: 562574
All target_characters
['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '\xa0', '£', '¦', '®', 'º', '»', '¿', 'â', 'é', 'ï', 'ñ', 'ó', 'ô', '–', '—', '‘', '’', '“', '”', '…', '€']
All input characters
[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Number of unique input tokens: 38
Number of unique output tokens: 114
Max sequence length for inputs: 1428
Max sequence length for outputs: 1493
Finished


In [ ]:
print(input_texts[:10])
print(target_texts[:10])

['chogm tangani kesan perubahan iklim ', 'kampala uganda 25 nov mesyuarat ketua ketua kerajaan komanwel chogm 2007 yang berakhir hari ini mendesak masyarakat antarabangsa melakukan sesuatu lebih konkrit menangani kesan ancaman perubahan iklim ', 'pada masa yang sama para pemimpin yang hadir mahukan negara maju membantu negara miskin mencapai matlamat pembangunan alaf baru mdg ', 'dua isu ini terkandung dalam dua kenyataan berasingan yang dikeluarkan pada akhir mesyuarat dwi tahunan selama tiga hari itu selain pernyataan bersama secara umum ', 'seramai 35 ketua kerajaan daripada 53 negara anggota komanwel menyertai chogm 2007 yang pada awalnya dibayangi isu penggantungan pakistan sehingga kelihatan menenggelam isu lain ', 'bagaimanapun komitmen negara anggota kali ini lebih matang dimana isu isu semasa turut di beri perhatian secara serius ', 'pada mesyuarat itu pemimpin komanwel mengesahkan komitmen berterusan kepada konvensyen rangka kerja pertubuhan bangsa bangsa bersatu mengenai per

In [ ]:
#added line of data from norm.text
with open(filepath + 'norm.txt', 'r',encoding='cp1252') as f:
  normalize_text_set = f.read().split('\n')  

print(normalize_text_set[:10])

['berkoban adalah satu bentuk ujian daripada allah s.w.t', 'kerana dalam menjalani kehidupan di dunia ini kita harus melakukan pengorbanan', 'hidup tanpa pengorbanan adalah kehidupan yang gagal', 'dalam menjalani hidup yang baik kita mesti berkorban masa dan tenaga misalnya berusaha mencari rezeki yang halal bagi mengelakkan kemiskinan', 'a merupakan tuntutan hidup kata pensyarah jabatan dakwah dan pembangunan insan akademi pengajian islam universiti malaya roslan mohamed mengenai konsep berkorban dalam islam', 'korban berasal daripada perkataan arab iaitu udhiyah', 'a menjurus kepada peristiwa pengorbanan satu keluarga iaitu nabi ibrahim dan isterinya hajar yang sanggup mengorbankan sesuatu yang amat mereka sayangi iaitu anaknya nabi ismail demi kasih dan cinta kepada allah s.w.t', 'ini adalah nilai pengorbanan yang cukup tinggi di sisi allah s.w.t', 'kerana ia bukan sahaja menguji kesabaran nabi ibrahim tetapi menduga sejauh mana ketakwaan nabi ibrahim', 'maka allah s.w.t menggantika

In [ ]:
with open(filepath + 'unnorm.txt', 'r',encoding='cp1252') as f:
  unnormalize_text_set = f.read().split('\n')

print(unnormalize_text_set[:10])

['"Berkoban adalah satu bentuk ujian daripada Allah s.w.t. ', 'kerana dalam menjalani kehidupan di dunia ini, kita harus melakukan pengorbanan. ', 'Hidup tanpa pengorbanan adalah kehidupan yang gagal.', 'Dalam menjalani hidup yang baik, kita mesti berkorban masa dan tenaga misalnya berusaha mencari rezeki yang halal bagi mengelakkan kemiskinan. ', 'a merupakan tuntutan hidup," kata pensyarah Jabatan Dakwah dan Pembangunan Insan, Akademi Pengajian Islam, Universiti Malaya, Roslan Mohamed mengenai konsep berkorban dalam Islam.', 'Korban berasal daripada perkataan Arab iaitu udhiyah.', 'a menjurus kepada peristiwa pengorbanan satu keluarga iaitu Nabi Ibrahim dan isterinya, Hajar yang sanggup mengorbankan sesuatu yang amat mereka sayangi iaitu anaknya, Nabi Ismail demi kasih dan cinta kepada Allah s.w.t.', '"Ini adalah nilai pengorbanan yang cukup tinggi di sisi Allah s.w.t. ', 'kerana ia bukan sahaja menguji kesabaran Nabi Ibrahim tetapi menduga sejauh mana ketakwaan Nabi Ibrahim. ', 'Mak

In [ ]:
# Load parameters
with open(filepath + 'params_char.pkl', "rb") as file:
    parameters = pickle.load(file)

In [ ]:
# function to generate input text one-hot sequence
# in one hot sequence x-axis indicates unique charater value and y-axis indicates number in sequence
def get_text_encodings_generate(texts):
    enc_seq = parameters["enc_token"].texts_to_sequences(texts)
    pad_seq = pad_sequences(enc_seq, maxlen=parameters["max_encoder_seq_length"],
                            padding='post')
    pad_seq = to_categorical(pad_seq, num_classes=parameters["num_encoder_tokens"])
    return pad_seq

In [ ]:
# function to generate target text one-hot sequence
def get_text_decodings_generate(texts):
    enc_seq = parameters["dec_token"].texts_to_sequences(texts)
    pad_seq = pad_sequences(enc_seq, maxlen=parameters["max_decoder_seq_length"],
                            padding='post')
    pad_seq = to_categorical(pad_seq, num_classes=parameters["num_decoder_tokens"])
  
    return pad_seq

In [ ]:
# function generate batch is to generate small batch size sequence of data in tensor to train
# this is because if np.zeros capture all the data from training data which has 562574 lines will cause the RAM to crash
def generate_batch(X, y, batch_size=128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            
            encoder_input_data = np.zeros((batch_size, parameters["max_encoder_seq_length"], parameters["num_encoder_tokens"]), dtype='float32')

            decoder_input_data = np.zeros((batch_size, parameters["max_decoder_seq_length"], parameters["num_decoder_tokens"]), dtype='float32')

            decoder_target_data = np.zeros((batch_size, parameters["max_decoder_seq_length"], parameters["num_decoder_tokens"]), dtype='float32')

            #for each line in tensor will input one hot sequence for encoder_input_data, decoder_input_data and decoder_target_data
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                
                encoder_input_data[i] = get_text_encodings_generate([input_text])
                decoder_input_data[i] = get_text_decodings_generate([target_text])
                decoder_target_data[i] = get_text_decodings_generate([target_text[1:]])

            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
#Generate model for this character-level project
encoder_inputs = Input(shape=(None,parameters["num_encoder_tokens"],))
encoder = Bidirectional(LSTM(128,return_sequences=True, return_state=True),merge_mode='concat')
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)

encoder_h = concatenate([forward_h, backward_h])
encoder_c = concatenate([forward_c, backward_c])

decoder_inputs = Input(shape=(None, parameters["num_decoder_tokens"],))
decoder_lstm = LSTM(256, return_sequences=True)
decoder_outputs = decoder_lstm(decoder_inputs, initial_state=[encoder_h, encoder_c])

attention = dot([decoder_outputs, encoder_outputs], axes=(2, 2))
attention = Activation('softmax', name='attention')(attention)
context = dot([attention, encoder_outputs], axes=(2, 1))
decoder_combined_context = concatenate([context, decoder_outputs])

output = TimeDistributed(Dense(128, activation="relu"))(decoder_combined_context)
output = TimeDistributed(Dense( parameters["num_decoder_tokens"], activation="softmax"))(output)

model = Model([encoder_inputs, decoder_inputs], [output])

In [ ]:
#print out the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 38)]   0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, None, 256),  171008      input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 114)]  0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 256)          0           bidirectional[0][1]              
                                                                 bidirectional[0][3]          

In [ ]:
#print out number of line of training data
print('input_texts length :' , len(input_texts))

NameError: ignored

In [ ]:
#function to train model
def train_model(batch_size = 128, epochs=5, input_texts=input_texts, target_texts=target_texts):

    #set up optimizer, loss function and metric for the model training
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    #start training with training data from input text and target text
    #validating the training using validation_data to test with testing data from normalize_text_set (norm.txt) and unnormalize_text_set (unnorm.txt)
    model.fit_generator(
        generator=generate_batch(X=input_texts, y=target_texts, batch_size=batch_size),
        steps_per_epoch=math.ceil(len(input_texts)/batch_size),
        epochs=epochs,
        verbose=1,
        validation_data = generate_batch(X=normalize_text_set, y=unnormalize_text_set, batch_size=batch_size),
        validation_steps=math.ceil(len(normalize_text_set)/batch_size),
        workers=1,
    )

    #save the weight from the model
    weights_path = os.path.join(filepath, 'checkpoint_char.h5')
    model.save_weights(weights_path)

In [ ]:
#train 5 epochs
train_model(epochs=5)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
4396/4396 [==============================] - 4131s 938ms/step - loss: 0.2480 - accuracy: 0.9406 - val_loss: 0.0305 - val_accuracy: 0.9836
Epoch 2/5
4396/4396 [==============================] - 4093s 931ms/step - loss: 0.0091 - accuracy: 0.9974 - val_loss: 0.0336 - val_accuracy: 0.9543
Epoch 3/5
4396/4396 [==============================] - 4086s 929ms/step - loss: 0.0062 - accuracy: 0.9982 - val_loss: 0.0344 - val_accuracy: 0.9918
Epoch 4/5
4396/4396 [==============================] - 4086s 930ms/step - loss: 0.0047 - accuracy: 0.9986 - val_loss: 0.0359 - val_accuracy: 0.9546
Epoch 5/5
4396/4396 [==============================] - 4099s 932ms/step - loss: 0.0040 - accuracy: 0.9988 - val_loss: 0.0404 - val_accuracy: 0.9542


In [ ]:
#load the weight in the model
weights_path = os.path.join(filepath, 'checkpoint_char.h5')
model.load_weights(weights_path)

In [ ]:
#decode the text
def decode(model, input_text, batch_size=128):
    #process the input data
    input_text = input_text.lower() 
    input_text = re.sub(r"[^a-zA-Z0-9]", " ", input_text)
    input_text=re.sub(r'[ ]{2,}',r' ',input_text) 

    #put the text in a list
    input_text_list = [input_text]

    #change to encoder input data one hot sequence
    input_sequences = get_text_encodings_generate(input_text_list)
    parameters["reverse_dec_dict"][0] = "\n"
    outputs = [""] * len(input_sequences)

    #contruct decoder input data one hot sequence
    target_text = "\t"
    target_seq = parameters["dec_token"].texts_to_sequences([target_text] * len(input_sequences))
    target_seq = pad_sequences(target_seq, maxlen=parameters["max_decoder_seq_length"],
                               padding="post")
    target_seq_hot = to_categorical(target_seq, num_classes=parameters["num_decoder_tokens"])
    
    extra_char_count = [0] 
    i = 0
    while True:
        curr_char_index = [i - extra_char_count[0]]
        input_encodings = np.argmax(input_sequences, axis=2)
        cur_inp_list = [input_encodings[0][curr_char_index[0]]]

        #predict and change the sequence to words
        output_tokens = model.predict([input_sequences, target_seq_hot], batch_size=batch_size)
        sampled_possible_indices = np.argsort(output_tokens[:, i, :])[:, ::-1].tolist()
        sampled_token_indices = []
        for j, per_char_list in enumerate(sampled_possible_indices):
            for index in per_char_list:
                if parameters["reverse_dec_dict"][index] == '\n' and cur_inp_list[j] != 0:
                    continue
                sampled_token_indices.append(index)
                break

        sampled_chars = [parameters["reverse_dec_dict"][index] for index in sampled_token_indices]
        output_text=""
        outputs = [outputs[j] + sampled_chars[j] for j, output in enumerate(outputs)]
        end_indices = sorted([index for index, char in enumerate(sampled_chars) if char == '\n'], reverse=True)
        
        for index in end_indices:
            output_text=outputs[index].strip()

        #if end_indices has value 
        if len(end_indices)!=0:
            break
        target_seq[:, i + 1] = sampled_token_indices
        target_seq_hot = to_categorical(target_seq, num_classes=parameters["num_decoder_tokens"])
        i += 1
    return output_text

In [ ]:
print(decode(model, "dalam kejadian itu muhammad israf izzuddin sembilan belas yang menunggang motosikal dirempuh kereta dipandu lelaki itu yang berlaku pada pukul dua belas tengah malam"))

Dalam kejadian itu, Muhammad Israf Izzuddin sembilan belas yang menunggang motosikal dirempuh kereta dipandu lelaki itu yang berlaku pada pukul dua belas tengah malam


In [ ]:
with open(filepath + 'testset.txt', 'r',encoding='cp1252') as f:
  test_data = f.read().split('\n')

for line in test_data:
    print("Original Text:" + line)
    line = line.lower() 
    line = re.sub(r"[^a-zA-Z0-9]", " ", line)
    line=re.sub(r'[ ]{2,}',r' ',line) 
    print("Normalized Text:" + line)
    print("Predicted Text:" + decode(model, line))
    print()